In [1]:
import cv2 as cv
import numpy as np

src = cv.imread("plant001.png")
cv.imshow('Source Image', src)
cv.waitKey(0)
cv.destroyAllWindows()

D:\Anaconda3\envs\ProjectEnv\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda3\envs\ProjectEnv\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
D:\Anaconda3\envs\ProjectEnv\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
hsv_src = cv.cvtColor(src, cv.COLOR_BGR2HSV)

cv.imshow('HSV Image', hsv_src)
cv.waitKey(0)
cv.destroyAllWindows()

In [3]:
mask = cv.inRange(hsv_src, (35, 50, 50), (85, 255,255))


cv.imshow('Mask', mask)
cv.waitKey(0)
cv.destroyAllWindows()

In [5]:
masked_img = cv.bitwise_and(hsv_src, hsv_src, mask=mask)

cv.imshow('Bitwise', masked_img)
cv.waitKey(0)
cv.destroyAllWindows()

In [6]:
blurred = cv.GaussianBlur(masked_img, (5, 5), 0)

cv.imshow('Source Image', blurred)
cv.waitKey(0)
cv.destroyAllWindows()

green_only = blurred

In [7]:
coverted = cv.cvtColor(green_only, cv.COLOR_HSV2BGR)

coverted[:,:,0] = 0
coverted[:,:,2] = 0

coverted = cv.medianBlur(coverted, 5)

cv.imshow('Binary Image', coverted)
cv.waitKey(0)
cv.destroyAllWindows()

In [13]:
#continue here
imgLaplacian = cv.Laplacian(coverted, -1, 3, 3)

sharp = np.float32(coverted)

# temp = cv.addWeighted(sharp, 1.5, imgLaplacian, -0.5, 0)
imgResult = sharp - imgLaplacian


# convert back to 8bits gray scale
imgResult = np.clip(imgResult, 0, 255)
imgResult = imgResult.astype('uint8')

imgLaplacian = np.clip(imgLaplacian, 0, 255)
imgLaplacian = np.uint8(imgLaplacian)

cv.imshow('Laplace Filtered Image', imgLaplacian)
cv.imshow('New Sharped Image', imgResult)

cv.waitKey(0)
cv.destroyAllWindows()

In [14]:
bw = imgResult[:,:,1] #cv.cvtColor(imgResult, cv.COLOR_BGR2GRAY)

# edges = cv.Canny(bw, 100,200)
# bw = bw + 1*edges

cv.imshow('Binary Image', bw)
cv.waitKey(0)
cv.destroyAllWindows()

In [17]:
ret, dst = cv.threshold(bw, 50,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
element = cv.getStructuringElement(cv.MORPH_RECT, (7,7))
dst = cv.morphologyEx(dst, cv.MORPH_OPEN, element)

cv.imshow('Binary Image', dst)
cv.waitKey(0)
cv.destroyAllWindows()

In [23]:
# Perform the distance transform algorithm
dist = cv.distanceTransform(dst, cv.DIST_L2, 3)
# Normalize the distance image for range = {0.0, 1.0}
# so we can visualize and threshold it
cv.normalize(dist, dist, 0, 1.0, cv.NORM_MINMAX)
cv.imshow('Distance Transform Image', dist)
cv.waitKey(0)
cv.destroyAllWindows()

In [24]:
# Threshold to obtain the peaks
# This will be the markers for the foreground objects
_, dist = cv.threshold(dist, 0.5*dist.max(), 255, 0)
# Dilate a bit the dist image
# kernel1 = np.ones((3,3), dtype=np.uint8)
# dist = cv.dilate(dist, kernel1)
cv.imshow('Peaks', dist)
cv.waitKey(0)
cv.destroyAllWindows()

In [13]:
# Create the CV_8U version of the distance image
# It is needed for findContours()
dist_8u = dist.astype('uint8')
# Find total markers
contours, _ = cv.findContours(dist_8u, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
# Create the marker image for the watershed algorithm
markers = np.zeros(dist.shape, dtype=np.int32)
# Draw the foreground markers
for i in range(len(contours)):
    cv.drawContours(markers, contours, i, (i+1), -1)

# Draw the background marker
cv.circle(markers, (5,5), 3, (255,255,255), -1)

markers_8u = (markers * 50).astype('uint8')
# cv.imshow('Markers', markers_8u)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [14]:
import random as rng

# Perform the watershed algorithm
cv.watershed(imgResult, markers)
#mark = np.zeros(markers.shape, dtype=np.uint8)
mark = markers.astype('uint8')
mark = cv.bitwise_not(mark)
# uncomment this if you want to see how the mark
# image looks like at that point
#cv.imshow('Markers_v2', mark)
# Generate random colors
colors = []
for contour in contours:
    colors.append((rng.randint(0,256), rng.randint(0,256), rng.randint(0,256)))
# Create the result image
dst = np.zeros((markers.shape[0], markers.shape[1], 3), dtype=np.uint8)
# Fill labeled objects with random colors
for i in range(markers.shape[0]):
    for j in range(markers.shape[1]):
        index = markers[i,j]
        if index > 0 and index <= len(contours):
            dst[i,j,:] = colors[index-1]
            
# Visualize the final image
# cv.imshow('Final Result', dst)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [15]:
alpha = 0.5
beta = (1.0 - alpha)
out = cv.addWeighted(src, alpha, dst, beta, 50)
# [blend_images]
# [display]
cv.imshow('dst', out)
cv.waitKey(0)

# [display]
cv.destroyAllWindows()